In [1]:
import pandas as pd
import numpy as np
import json
import gzip as gunzip
from matplotlib import pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 5000)
from datetime import datetime
import pytz
import tarfile

In [2]:
print(datetime.today().strftime("%d-%b-%Y %a"), "Notebook")
tz_Suisse = pytz.timezone('Europe/Zurich')
current_time= datetime.now(tz_Suisse)
print("Lausanne Time : ", current_time.strftime("%H:%M:%S"))

25-Oct-2022 Tue Notebook
Lausanne Time :  02:35:14


## You should change the datapth that includes your path to use csv files

In [5]:
train_df = pd.read_csv(datapath + "train.csv")
train_df.head()


,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,DER_lep_eta_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,100000,s,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,41.928,197.760,1.582,1.396,0.2,32.638,1.017,0.381,51.626,2.273,-2.414,16.824,-0.277,258.733,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497
1,100001,b,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,2.078,125.157,0.879,1.414,-999.0,42.014,2.039,-3.011,36.918,0.501,0.103,44.704,-1.916,164.546,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226
2,100002,b,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,9.336,197.814,3.776,1.414,-999.0,32.154,-0.705,-2.093,121.409,-0.953,1.052,54.283,-2.186,260.414,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251
3,100003,b,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,0.414,75.968,2.354,-1.285,-999.0,22.647,-1.655,0.010,53.321,-0.522,-3.100,31.082,0.060,86.062,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000
4,100004,b,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,16.405,57.983,1.056,-1.385,-999.0,28.209,-2.197,-2.231,29.774,0.798,1.569,2.723,-0.871,53.131,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000


In [6]:
train_df.shape

(250000, 32)

In [7]:
train_df.columns

Index(['Id', 'Prediction', 'DER_mass_MMC', 'DER_mass_transverse_met_lep',
       'DER_mass_vis', 'DER_pt_h', 'DER_deltaeta_jet_jet', 'DER_mass_jet_jet',
       'DER_prodeta_jet_jet', 'DER_deltar_tau_lep', 'DER_pt_tot', 'DER_sum_pt',
       'DER_pt_ratio_lep_tau', 'DER_met_phi_centrality',
       'DER_lep_eta_centrality', 'PRI_tau_pt', 'PRI_tau_eta', 'PRI_tau_phi',
       'PRI_lep_pt', 'PRI_lep_eta', 'PRI_lep_phi', 'PRI_met', 'PRI_met_phi',
       'PRI_met_sumet', 'PRI_jet_num', 'PRI_jet_leading_pt',
       'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt',
       'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_all_pt'],
      dtype='object')

In [8]:
train_df['PRI_jet_num'].value_counts()

0    99913
1    77544
2    50379
3    22164
Name: PRI_jet_num, dtype: int64

In [9]:
train_df.describe()

,Id,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,DER_lep_eta_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,224999.500000,-49.023079,49.239819,81.181982,57.895962,-708.420675,-601.237051,-709.356603,2.373100,18.917332,158.432217,1.437609,-0.128305,-708.985189,38.707419,-0.010973,-0.008171,46.660207,-0.019507,0.043543,41.717235,-0.010119,209.797178,0.979176,-348.329567,-399.254314,-399.259788,-692.381204,-709.121609,-709.118631,73.064591
std,72168.927986,406.345647,35.344886,40.828691,63.655682,454.480565,657.972302,453.019877,0.782911,22.273494,115.706115,0.844743,1.193585,453.596721,22.412081,1.214079,1.816763,22.064922,1.264982,1.816611,32.894693,1.812223,126.499506,0.977426,532.962789,489.338286,489.333883,479.875496,453.384624,453.389017,98.015662
min,100000.000000,-999.000000,0.000000,6.329000,0.000000,-999.000000,-999.000000,-999.000000,0.208000,0.000000,46.104000,0.047000,-1.414000,-999.000000,20.000000,-2.499000,-3.142000,26.000000,-2.505000,-3.142000,0.109000,-3.142000,13.678000,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000
25%,162499.750000,78.100750,19.241000,59.388750,14.068750,-999.000000,-999.000000,-999.000000,1.810000,2.841000,77.550000,0.883000,-1.371000,-999.000000,24.591750,-0.925000,-1.575000,32.375000,-1.014000,-1.522000,21.398000,-1.575000,123.017500,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000
50%,224999.500000,105.012000,46.524000,73.752000,38.467500,-999.000000,-999.000000,-999.000000,2.491500,12.315500,120.664500,1.280000,-0.356000,-999.000000,31.804000,-0.023000,-0.033000,40.516000,-0.045000,0.086000,34.802000,-0.024000,179.739000,1.000000,38.960000,-1.872000,-2.093000,-999.000000,-999.000000,-999.000000,40.512500
75%,287499.250000,130.606250,73.598000,92.259000,79.169000,0.490000,83.446000,-4.593000,2.961000,27.591000,200.478250,1.777000,1.225000,0.000000,45.017000,0.898000,1.565000,53.390000,0.959000,1.618000,51.895000,1.561000,263.379250,2.000000,75.349000,0.433000,0.503000,33.703000,-2.457000,-2.275000,109.933750
max,349999.000000,1192.026000,690.075000,1349.351000,2834.999000,8.503000,4974.979000,16.690000,5.684000,2834.999000,1852.462000,19.773000,1.414000,1.000000,764.408000,2.497000,3.142000,560.271000,2.503000,3.142000,2842.617000,3.142000,2003.976000,3.000000,1120.573000,4.499000,3.141000,721.456000,4.500000,3.142000,1633.433000


In [11]:
test_df = pd.read_csv(datapath + "test.csv")
test_df.head()

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,DER_lep_eta_centrality,PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,350000,?,-999.000,79.589,23.916,3.036,-999.000,-999.000,-999.000,0.903,3.036,56.018,1.536,-1.404,-999.000,22.088,-0.540,-0.609,33.930,-0.504,-1.511,48.509,2.022,98.556,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
1,350001,?,106.398,67.490,87.949,49.994,-999.000,-999.000,-999.000,2.048,2.679,132.865,1.777,-1.204,-999.000,30.716,-1.784,3.054,54.574,-0.169,1.795,21.093,-1.138,176.251,1,47.575,-0.553,-0.849,-999.000,-999.000,-999.000,47.575
2,350002,?,117.794,56.226,96.358,4.137,-999.000,-999.000,-999.000,2.755,4.137,97.600,1.096,-1.408,-999.000,46.564,-0.298,3.079,51.036,-0.548,0.336,19.461,-1.868,111.505,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
3,350003,?,135.861,30.604,97.288,9.104,-999.000,-999.000,-999.000,2.811,9.104,94.112,0.819,-1.382,-999.000,51.741,0.388,-1.408,42.371,-0.295,2.148,25.131,1.172,164.707,0,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,0.000
4,350004,?,74.159,82.772,58.731,89.646,1.347,536.663,-0.339,1.028,77.213,721.552,1.713,-0.913,0.004,45.087,-1.548,1.877,77.252,-1.913,2.838,22.200,-0.231,869.614,3,254.085,-1.013,-0.334,185.857,0.335,2.587,599.213


In [3]:
from helper_functions_project1 import *
from run import *
train_y, train_x, train_ids = load_csv_data("../../ML_EPFL/ml_project_dataset/train.csv")
test_y, test_x, test_ids = load_csv_data("../../ML_EPFL/ml_project_dataset/test.csv")

In [4]:
jet_number0 = np.where(train_x[:,22] == 0)
jet_number1 = np.where(train_x[:,22] == 1)
jet_number2 = np.where(train_x[:,22] == 2)
jet_number3 = np.where(train_x[:,22] == 3)

tr_jet_set0 = train_x[np.where(train_x[:,22] == 0)]
tr_jet_set1 = train_x[np.where(train_x[:,22] == 1)]
tr_jet_set2 = train_x[np.where(train_x[:,22] == 2)]
tr_jet_set3 = train_x[np.where(train_x[:,22] == 3)]

tr_y_jet_set0 = train_y[np.where(train_x[:,22] == 0)]
tr_y_jet_set1 = train_y[np.where(train_x[:,22] == 1)]
tr_y_jet_set2 = train_y[np.where(train_x[:,22] == 2)]
tr_y_jet_set3 = train_y[np.where(train_x[:,22] == 3)]

te_jet_set0 = test_x[np.where(test_x[:,22] == 0)]
te_jet_set1 = test_x[np.where(test_x[:,22] == 1)]
te_jet_set2 = test_x[np.where(test_x[:,22] == 2)]
te_jet_set3 = test_x[np.where(test_x[:,22] == 3)]

te_y_jet_set0 = test_y[np.where(test_x[:,22] == 0)]
te_y_jet_set1 = test_y[np.where(test_x[:,22] == 1)]
te_y_jet_set2 = test_y[np.where(test_x[:,22] == 2)]
te_y_jet_set3 = test_y[np.where(test_x[:,22] == 3)]

In [5]:
tr_jet_set0_deleted = np.delete(tr_jet_set0, [22], axis=1)
tr_jet_set1_deleted = np.delete(tr_jet_set1, [22], axis=1)
tr_jet_set2_deleted = np.delete(tr_jet_set2, [22], axis=1)
tr_jet_set3_deleted = np.delete(tr_jet_set3, [22], axis=1)

te_jet_set0_deleted = np.delete(te_jet_set0, [22], axis=1)
te_jet_set1_deleted = np.delete(te_jet_set1, [22], axis=1)
te_jet_set2_deleted = np.delete(te_jet_set2, [22], axis=1)
te_jet_set3_deleted = np.delete(te_jet_set3, [22], axis=1)

print(tr_jet_set0_deleted.shape)
print(tr_jet_set1_deleted.shape)
print(tr_jet_set2_deleted.shape)
print(tr_jet_set3_deleted.shape)
print(tr_y_jet_set0.shape)
print(tr_y_jet_set1.shape)
print(tr_y_jet_set2.shape)
print(tr_y_jet_set3.shape)
print(train_x.shape)
print(train_y.shape)

(99913, 29)
(77544, 29)
(50379, 29)
(22164, 29)
(99913,)
(77544,)
(50379,)
(22164,)
(250000, 30)
(250000,)


In [6]:
print(tr_jet_set0_deleted[0])
print(tr_y_jet_set0[0])

[ 1.43905e+02  8.14170e+01  8.09430e+01  4.14000e-01 -9.99000e+02
 -9.99000e+02 -9.99000e+02  3.31000e+00  4.14000e-01  7.59680e+01
  2.35400e+00 -1.28500e+00 -9.99000e+02  2.26470e+01 -1.65500e+00
  1.00000e-02  5.33210e+01 -5.22000e-01 -3.10000e+00  3.10820e+01
  6.00000e-02  8.60620e+01 -9.99000e+02 -9.99000e+02 -9.99000e+02
 -9.99000e+02 -9.99000e+02 -9.99000e+02  0.00000e+00]
-1.0


In [14]:
best_degree_selection(tr_jet_set0_deleted, tr_y_jet_set0, [5, 6,7,8], 5, [1e-3, 1e-4, 1e-5], seed = 353)

unnecessary columns are:  [4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28]
unnecessary columns are:  [4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28]
unnecessary columns are:  [4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28]
unnecessary columns are:  [4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28]
unnecessary columns are:  [4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28]
For lambda: 0.001 and degree:5, rmse_tr:0.7902956161428275, rmse_te:0.7929386174790707, acc_tr:0.8438569712741467, acc_te:0.8427885096586929
unnecessary columns are:  [4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28]
unnecessary columns are:  [4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28]
unnecessary columns are:  [4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28]
unnecessary columns are:  [4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28]
unnecessary columns are:  [4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28]
For lambda: 0.0001 and degree:5, rmse_tr:0.7902071390052285, rmse_te:0.7926335804993164, acc_tr:0.8438920028025223, acc_te:0.8429086177559804
unnecessary columns are:  [4, 5, 6, 1

((0.001, 6), 0.792486009513226, 0.8429686718046241)

In [15]:
best_degree_selection(tr_jet_set1_deleted, tr_y_jet_set1, [5,6,7,8], 5, [1e-3, 1e-4, 1e-5], seed = 353)

unnecessary columns are:  [4, 5, 6, 12, 25, 26, 27]
unnecessary columns are:  [4, 5, 6, 12, 25, 26, 27]
unnecessary columns are:  [4, 5, 6, 12, 25, 26, 27]
unnecessary columns are:  [4, 5, 6, 12, 25, 26, 27]
unnecessary columns are:  [4, 5, 6, 12, 25, 26, 27]
For lambda: 0.001 and degree:5, rmse_tr:0.8866698759466942, rmse_te:0.8891248111814928, acc_tr:0.8034530564869744, acc_te:0.8023471756512768
unnecessary columns are:  [4, 5, 6, 12, 25, 26, 27]
unnecessary columns are:  [4, 5, 6, 12, 25, 26, 27]
unnecessary columns are:  [4, 5, 6, 12, 25, 26, 27]
unnecessary columns are:  [4, 5, 6, 12, 25, 26, 27]
unnecessary columns are:  [4, 5, 6, 12, 25, 26, 27]
For lambda: 0.0001 and degree:5, rmse_tr:0.8864375836797512, rmse_te:0.8882602459005362, acc_tr:0.8035562290430744, acc_te:0.8027340727366521
unnecessary columns are:  [4, 5, 6, 12, 25, 26, 27]
unnecessary columns are:  [4, 5, 6, 12, 25, 26, 27]
unnecessary columns are:  [4, 5, 6, 12, 25, 26, 27]
unnecessary columns are:  [4, 5, 6, 12, 2

((0.0001, 8), 0.8779917402667371, 0.8072607686355429)

In [16]:
best_degree_selection(tr_jet_set2_deleted, tr_y_jet_set2, [5,6,7,8], 5, [2e-2/10, 1e-4, 1e-5], seed = 353)

unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
For lambda: 0.002 and degree:5, rmse_tr:0.8181313634132475, rmse_te:0.8221664047528492, acc_tr:0.8326650124069477, acc_te:0.8310074441687345
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
For lambda: 0.0001 and degree:5, rmse_tr:0.8172335165581627, rmse_te:0.8211961821052072, acc_tr:0.8330322580645161, acc_te:0.8314044665012407
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
For lambda: 1e-05 and degree:5, rmse_tr:0.8171483412130339, rmse_te:0.8209526190421803, acc_tr:0.8330669975186105, acc_te:0.8315037220843673
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are: 

((1e-05, 8), 0.8142993895347855, 0.8342233250620348)

In [17]:
best_degree_selection(tr_jet_set3_deleted, tr_y_jet_set3, [5,6,7,8], 5, [1e-3, 1e-4, 1e-5], seed = 353)

unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
For lambda: 0.001 and degree:5, rmse_tr:0.8179716686224878, rmse_te:0.826334083060815, acc_tr:0.8327278880866427, acc_te:0.8292418772563176
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
For lambda: 0.0001 and degree:5, rmse_tr:0.8181078569132255, rmse_te:0.8281903100419488, acc_tr:0.8326714801444043, acc_te:0.8284747292418772
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
For lambda: 1e-05 and degree:5, rmse_tr:0.8185494570380669, rmse_te:0.8286153744480849, acc_tr:0.8324909747292418, acc_te:0.8282942238267147
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  []
unnecessary columns are:  

((0.001, 7), 0.8130515051928981, 0.8346570397111913)

In [7]:
#train_y, train_x, train_ids = load_csv_data("../../ML_EPFL/ml_project_dataset/train.csv")
#test_y, test_x, test_ids = load_csv_data("../../ML_EPFL/ml_project_dataset/test.csv")

degree=6
# preprocessing
train_x0, test_x0 = apply_preprocessing(tr_jet_set0_deleted, te_jet_set0_deleted, degree=degree)
degree=8
train_x1, test_x1 = apply_preprocessing(tr_jet_set1_deleted, te_jet_set1_deleted, degree=degree)
degree=8
train_x2, test_x2 = apply_preprocessing(tr_jet_set2_deleted, te_jet_set2_deleted, degree=degree)
degree=7
train_x3, test_x3 = apply_preprocessing(tr_jet_set3_deleted, te_jet_set3_deleted, degree=degree)

unnecessary columns are:  [4, 5, 6, 12, 22, 23, 24, 25, 26, 27, 28]
unnecessary columns are:  [4, 5, 6, 12, 25, 26, 27]
unnecessary columns are:  []
unnecessary columns are:  []


In [8]:
# ridge regression
lambda_=0.001
w, _ = ridge_regression(tr_y_jet_set0.reshape(-1), train_x0, lambda_)

# train loss and accuracy calculation
preds_tr = train_x0.dot(w)
preds_tr[preds_tr < 0] = -1
preds_tr[preds_tr >= 0] = 1
loss_tr = np.sqrt(2 * compute_mse(tr_y_jet_set0.reshape(-1) - preds_tr))
acc_tr = np.sum(preds_tr == tr_y_jet_set0) / len(tr_y_jet_set0)

# test predictions
preds_te0 = test_x0.dot(w)
preds_te0[preds_te0 < 0] = -1
preds_te0[preds_te0 >= 0] = 1

In [9]:
# ridge regression
lambda_=0.0001
w, _ = ridge_regression(tr_y_jet_set1.reshape(-1), train_x1, lambda_)

# test predictions
preds_te1 = test_x1.dot(w)
preds_te1[preds_te1 < 0] = -1
preds_te1[preds_te1 >= 0] = 1

In [10]:
# ridge regression
lambda_=0.00001
w, _ = ridge_regression(tr_y_jet_set2.reshape(-1), train_x2, lambda_)

# test predictions
preds_te2 = test_x2.dot(w)
preds_te2[preds_te2 < 0] = -1
preds_te2[preds_te2 >= 0] = 1

In [11]:
# ridge regression
lambda_=0.001
w, _ = ridge_regression(tr_y_jet_set3.reshape(-1), train_x3, lambda_)

# test predictions
preds_te3 = test_x3.dot(w)
preds_te3[preds_te3 < 0] = -1
preds_te3[preds_te3 >= 0] = 1

In [12]:
pred_final = np.zeros(test_y.shape)
pred_final.shape

(568238,)

In [13]:
pred_final[np.where(test_x[:,22] == 0)] = preds_te0
print(np.sum(pred_final == 0))
pred_final[np.where(test_x[:,22] == 1)] = preds_te1
pred_final[np.where(test_x[:,22] == 2)] = preds_te2
pred_final[np.where(test_x[:,22] == 3)] = preds_te3
print(np.sum(pred_final == 0))

340780
0


In [14]:
create_csv_submission(test_ids, pred_final, "third_test.csv")